## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-11-02-02-20-59 +0000,startribune,"With election day ahead, Twin Cities mayoral c...",https://www.startribune.com/with-election-day-...
1,2025-11-02-02-05-47 +0000,nypost,Syracuse University chancellor says anti-Israe...,https://nypost.com/2025/11/01/us-news/prominen...
2,2025-11-02-02-00-00 +0000,wsj,China Is Filling Up Its Oil Reserves Fast,https://www.wsj.com/business/energy-oil/china-...
3,2025-11-02-01-55-07 +0000,nyt,"Zoë Wicomb, Acclaimed South African Author, Di...",https://www.nytimes.com/2025/10/31/books/zoe-w...
4,2025-11-02-01-47-40 +0000,nyt,Can Paul Atkins ‘Make I.P.O.s Great Again’?,https://www.nytimes.com/2025/11/01/business/de...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2343/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
42,trump,27
219,new,9
0,election,9
382,he,8
62,police,7


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
41,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...,56
156,2025-11-01-08-43-08 +0000,nyt,Carney Says He Apologized to Trump Over Ad Rev...,https://www.nytimes.com/2025/11/01/world/canad...,55
101,2025-11-01-14-00-00 +0000,wsj,Canadian PM Carney Says He Apologized to Trump...,https://www.wsj.com/world/canadian-pm-carney-s...,54
53,2025-11-01-20-44-56 +0000,bbc,US judges say Trump administration must contin...,https://www.bbc.com/news/articles/cr433x9zqq4o...,53
73,2025-11-01-18-25-24 +0000,nypost,Nicki Minaj sends lefty fans into meltdown aft...,https://nypost.com/2025/11/01/us-news/nicki-mi...,51


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
41,56,2025-11-01-21-48-00 +0000,nypost,"Trump to host Ahmad al-Sharaa, marking first-e...",https://nypost.com/2025/11/01/us-news/trump-to...
47,42,2025-11-01-20-55-31 +0000,nypost,House Republicans exploring ways to prevent Ma...,https://nypost.com/2025/11/01/us-news/house-re...
52,38,2025-11-01-20-45-56 +0000,nypost,"Great white sharks swarmed New York, New Jerse...",https://nypost.com/2025/11/01/us-news/great-wh...
43,32,2025-11-01-21-27-14 +0000,latimes,"As SNAP benefits lapse, thousands show up to S...",https://www.latimes.com/california/story/2025-...
87,27,2025-11-01-15-26-45 +0000,nyt,"In One Jamaican Town, Hurricane Melissa Left B...",https://www.nytimes.com/2025/10/31/world/ameri...
9,25,2025-11-02-01-28-51 +0000,nypost,Obama calls Zohran Mamdani and offers to be ‘s...,https://nypost.com/2025/11/01/us-news/obama-ha...
22,25,2025-11-01-23-39-02 +0000,bbc,300 million tourists just visited China's stun...,https://www.bbc.com/news/articles/c8x1rnnd5gjo...
120,25,2025-11-01-11-00-00 +0000,latimes,Man serving life in prison admits to three col...,https://www.latimes.com/california/story/2025-...
42,24,2025-11-01-21-38-02 +0000,wapo,Two additional suspects charged in Louvre jewe...,https://www.washingtonpost.com/world/2025/11/0...
73,24,2025-11-01-18-25-24 +0000,nypost,Nicki Minaj sends lefty fans into meltdown aft...,https://nypost.com/2025/11/01/us-news/nicki-mi...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
